In [1]:
spark.conf.set("spark.sql.shuffle.partitions", 1000)
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

In [2]:
from pyspark.ml.clustering import KMeans, KMeansModel, BisectingKMeans, BisectingKMeansModel
from pyspark.ml.feature import VectorAssembler, StandardScaler, PCA, PCAModel, ElementwiseProduct
from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql.functions import col, sum, greatest, expr, count, countDistinct, max, min, variance, bround, avg, lit, udf
from pyspark.sql import Row
import pandas as pd
import scipy.stats as stats
import matplotlib.pylab as plt
from pyspark.ml.linalg import Vectors
from pyspark.ml.linalg import SparseVector, DenseVector

#PARA EL TRATAMIENTO DE OUTLIERS
from pyspark.sql.functions import col, count, when
from functools import reduce
from operator import and_

/opt/conda/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/opt/conda/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')



### Data Importation

In [3]:
dfEM=spark.read.parquet("/analytics/bc1_clustering/tables/dfEM")

In [4]:
#dfBufferOrig=spark.read.parquet("/analytics/bc1_clustering/tables/dfBufferAccYrGrpPart")
#dfBufferOrig=spark.read.parquet("/analytics/bc1_clustering/tables/dfBufferAccYrSubPart2")
# dfBufferOrig=spark.read.parquet("/analytics/bc1_clustering/tables/dfBufferAccYrGrp")
# dfBufferOrig=spark.read.parquet("/analytics/bc1_clustering/tables/dfFinalMatrix11Dic")
# dfBufferOrig=spark.read.parquet("/analytics/bc1_clustering/tables/dfFinalMatrixGrpVal19Dic")
dfBufferOrig=spark.read.parquet("/analytics/bc1_clustering/tables/dfFinalMatrixGrpVal02Ene")

# dfBufferOrig=dfBufferOrig.withColumnRenamed('','NULO')

In [5]:
# dfBufferOrig.persist()

nameList=dfBufferOrig.schema.names[1:]

len(nameList)

727

In [ ]:
dfBufferOrig.count()

In [ ]:
# [dfBuffer, dfTestBuffer] = dfBufferOrig.randomSplit([0.5, 0.5],666)

In [6]:
dfBuffer=dfBufferOrig.where(~dfBufferOrig['ACCOUNTPK'].cast('string').substr(-1,1).isin(['0','1','2','3','4'])).persist()

In [7]:
dfTestBuffer=dfBufferOrig.where(dfBufferOrig['ACCOUNTPK'].cast('string').substr(-1,1).isin(['0','1','2','3','4'])).persist()

In [ ]:
#dfBuffer.write.parquet("/analytics/bc1_clustering/tables/dfTrainBufferBin")

In [ ]:
#dfTestBuffer.write.parquet("/analytics/bc1_clustering/tables/dfTestBufferBin")

In [ ]:
# dfBuffer=spark.read.parquet("/analytics/bc1_clustering/tables/dfTrainBufferBin")
# dfBuffer.persist()
# nameList=dfBuffer.schema.names[1:]

In [ ]:
# dfTestBuffer=spark.read.parquet("/analytics/bc1_clustering/tables/dfTestBufferBin")
# dfTestBuffer.persist()

In [8]:
dfBuffer.count()

1980853

In [9]:
dfTestBuffer.count()

2010927

In [ ]:
#dfTestBuffer.select(variance(dfTestBuffer.columns[2])).show()

In [ ]:
#dfBufferOrig.select(dfBufferOrig['ACCOUNTPK'].cast('string').substr(-1,1).alias('caracter')).groupBy('caracter').count().show()

In [ ]:
# dfTsys=spark.read.parquet("/analytics/bc1_clustering/tables/dfTsysAccYrOriginal")

In [ ]:
# dfTsys.persist()

In [10]:
dfTsys=dfBufferOrig.select('ACCOUNTPK','Emprestimo_Pessoal','Pagamento_de_Contas','Parcelamento_Fatura_Automatico','Parcelamento_Fatura_Facil','Saque_Parcelado','Saque_a_Vista','Overlimit')

### Binary matrix creation

In [ ]:
dfBufferBinary=dfBufferOrig.select([ i if i=='ACCOUNTPK' else when(col(i)!=0,1).otherwise(0).alias(i) for i in dfBufferOrig.columns])

In [ ]:
dfBufferBinary.persist().count()

In [ ]:
dfBufferBinary.write.parquet("/analytics/bc1_clustering/tables/dfBufferAccYrSubPartBin2")

In [ ]:
# #TEST CONVERSOR A MATRIZ 0/1
# l = [('A',2.0,-3.0,0,-30.0),('B',0,-300.0,1,0),('C',7.0,0.0,2,-50.0),('D',12.0,-700.0,0,0)]
# rdd = sc.parallelize(l)
# dummy = rdd.map(lambda x: Row(ID=x[0], col1=float(x[1]),col2=float(x[2]),col3=float(x[3]),col4=float(x[4])))
# dummyDf = sqlContext.createDataFrame(dummy)
# dummyDf.show(5)
# dummyDf.select([ i if i=='ID' else when(col(i)!=0,1).otherwise(0).alias(i) for i in dummyDf.columns]).show()

### Variable distribution histogram

In [ ]:
#METODO HISTOGRAM DF
plt.clf()
#bins, counts = dummyDf.select('col1').rdd.flatMap(lambda x: x).histogram(20)
bins, counts = dfBuffer.select('CREDITLIMIT').rdd.flatMap(lambda x: x).histogram(20)
plt.hist(bins[:-1], bins=bins, weights=counts)

plt.show()
#.filter(col('101054')>0)

In [ ]:
# #METODO hist DE PLT, parece que no funciona con grandes cantidades de datos
# plt.clf()
# #n, bins, patches = plt.hist(dummyDf.select('col4').collect(), 15, normed=1, facecolor='green', alpha=0.75)
# n, bins, patches = plt.hist(dfBuffer.select('1010540800').collect(), 15, normed=1, facecolor='green', alpha=0.75)
# plt.show()

### Outlier treatment (for money value matrices)

In [ ]:
len(nameList)

In [ ]:
listaCols=[]
listaLimit=[]

for i in nameList:
    cuenta=dfBuffer.filter(col(i)>0).count()
    if cuenta>0:
        listaCols.append(i)
        Q1, Q3 = dfBuffer.filter(col(i)>0).approxQuantile(col=i,probabilities=[0.25,0.75],relativeError=0.01)
        IQR=(Q3-Q1)
        top_limit=Q3+9*IQR
        listaLimit.append(top_limit)

In [ ]:
expr=reduce(and_, [col(c) < t for c, t in zip(listaCols, listaLimit)])

In [ ]:
dfBufferOutlier=dfBuffer.where(expr)

In [ ]:
dfBufferOutlier.persist().count()

In [ ]:
dfBufferOutlier.write.parquet("/analytics/bc1_clustering/tables/dfBufferOutlier")

In [ ]:
dfBufferOutlier=spark.read.parquet("/analytics/bc1_clustering/tables/dfBufferOutlier")

In [ ]:
dfBufferOutlier.count()

### Feature Assembler and Scaler

#### Binary matrices

In [ ]:
dfBufferOutlier=dfBuffer
#dfBufferOutlier=dfTestBuffer.select(dfTestBuffer.columns[0:2400])

In [ ]:
#continuous_feature_assembler= VectorAssembler(inputCols=dfTestBuffer.columns[0:2400], outputCol="features")
continuous_feature_assembler= VectorAssembler(inputCols=nameList, outputCol="features")

In [ ]:
estimatorsKmeans = [continuous_feature_assembler]

featurePipeline = Pipeline(stages=estimatorsKmeans)

sparkFeaturePipelineModel = featurePipeline.fit(dfBufferOutlier)

In [ ]:
dfBufferFeature=sparkFeaturePipelineModel.transform(dfBufferOutlier).persist()

In [ ]:
dfBufferFeature.count()

#### Money value matrices

In [11]:
#SI NO SE EJECUTA EL PROCESO DE OUTLIERS
dfBufferOutlier=dfBuffer
#dfBufferOutlier=dfTestBuffer.select(dfTestBuffer.columns[0:2400])

In [12]:
nameList=dfBuffer.schema.names[1:]

In [13]:
continuous_feature_assembler= VectorAssembler(inputCols=nameList, outputCol="unscaled_continuous_features")

In [14]:
continuous_feature_scaler = StandardScaler(inputCol="unscaled_continuous_features", outputCol="scaled_features",\
                                           withStd=True, withMean=True)

In [15]:
estimatorsKmeans = [continuous_feature_assembler, continuous_feature_scaler]

featurePipeline = Pipeline(stages=estimatorsKmeans)

sparkFeaturePipelineModel = featurePipeline.fit(dfBufferOutlier)

In [16]:
dfBufferScaled=sparkFeaturePipelineModel.transform(dfBufferOutlier).persist()

In [17]:
dfBufferScaled.count()

1980853

In [18]:
dfBufferScaled.printSchema()

root
 |-- ACCOUNTPK: long (nullable = true)
 |-- BALANCE: double (nullable = true)
 |-- INSTALMENTBALANCEFOROVLMCHECK: double (nullable = true)
 |-- CREDITLIMIT: double (nullable = true)
 |-- LimitUse: double (nullable = true)
 |-- LimitOffUs: double (nullable = true)
 |-- LimitPersLoan: double (nullable = true)
 |-- LimitCashOut: double (nullable = true)
 |-- CLOSING_BALANCE: double (nullable = true)
 |-- INSTALLMENTSOFFERS: long (nullable = true)
 |-- Emprestimo_Pessoal: long (nullable = true)
 |-- Pagamento_de_Contas: long (nullable = true)
 |-- Parcelamento_Fatura_Automatico: long (nullable = true)
 |-- Parcelamento_Fatura_Facil: long (nullable = true)
 |-- Saque_Parcelado: long (nullable = true)
 |-- Saque_a_Vista: long (nullable = true)
 |-- Overlimit: long (nullable = true)
 |-- Overlimit_Auth: integer (nullable = true)
 |-- NULO: double (nullable = true)
 |-- 0742: double (nullable = true)
 |-- 0763: double (nullable = true)
 |-- 0780: double (nullable = true)
 |-- 1520: double

#### Increase of the imporance of the financial variables (money)

In [19]:
listaPesos=([1]*727)
#listaPesos[0:17]=[10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10]
VectorPesos=DenseVector(listaPesos)

In [20]:
transformer = ElementwiseProduct(scalingVec=VectorPesos,
                                 inputCol="scaled_features", outputCol="features")
# Batch transform the vectors to create new column:
dfBufferFeature=transformer.transform(dfBufferScaled)

In [21]:
dfBufferFeature.select('scaled_features','features').show(1,truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### Dimensionality reduction (WIP)

In [ ]:
pca = PCA(k=120, inputCol="features", outputCol="pca_features")

In [ ]:
pcamodel = pca.fit(dfBufferFeature)

In [ ]:
variance=pcamodel.explainedVariance

In [ ]:
variance

In [ ]:
variance.sum()

In [ ]:
dfBufferFeaturePCA=pcamodel.transform(dfBufferFeature)

### K Evaluation

In [ ]:
listaErrores=[]

for k in range(2,51):
    #print(k)
    kmeans = KMeans().setK(k).setSeed(666)
    #kmeans = BisectingKMeans().setK(k).setSeed(666)
    model = kmeans.fit(dfBufferFeature)
    wssse = model.computeCost(dfBufferFeature)
    print("Within Set Sum of Squared Errors {} = ".format(k) + str(wssse))
    listaErrores.append((k,wssse))

In [ ]:
pdErrors=pd.DataFrame.from_records(listaErrores, columns=['clusters','wssse'])

In [ ]:
plt.clf()
plt.plot(pdErrors.clusters,pdErrors.wssse)
plt.show()

In [ ]:
dfErrors=spark.createDataFrame(pdErrors)

In [ ]:
dfErrors.printSchema()

In [ ]:
# dfErrors.write.parquet("/analytics/bc1_clustering/tables/dfErrorsBin")

### Segmentation final model

In [25]:
kmeans = KMeans().setK(20).setSeed(666)
# kmeans = BisectingKMeans().setK(20).setMinDivisibleClusterSize(40000).setSeed(666)

In [26]:
dfBufferFeature.select("ACCOUNTPK","features").show(5)

+---------+--------------------+
|ACCOUNTPK|            features|
+---------+--------------------+
|    67805|[0.53609270966588...|
|   109366|[0.53609270966588...|
|   140126|[-3.4885453062219...|
|   142317|[-0.1599475963050...|
|   164347|[0.53609270966588...|
+---------+--------------------+
only showing top 5 rows



In [27]:
model = kmeans.fit(dfBufferFeature)

### Financial Product Inclination

#### Train

In [28]:
dfTrainRes=model.transform(dfBufferFeature).select(col("ACCOUNTPK").alias("ACCOUNT"),"prediction")

In [29]:
dfProdIncl=dfTrainRes.join(dfBuffer,dfTrainRes.ACCOUNT==dfTsys.ACCOUNTPK,'left_outer').fillna(0)

In [30]:
dfProdIncl.printSchema()

root
 |-- ACCOUNT: long (nullable = true)
 |-- prediction: integer (nullable = true)
 |-- ACCOUNTPK: long (nullable = true)
 |-- BALANCE: double (nullable = false)
 |-- INSTALMENTBALANCEFOROVLMCHECK: double (nullable = false)
 |-- CREDITLIMIT: double (nullable = false)
 |-- LimitUse: double (nullable = false)
 |-- LimitOffUs: double (nullable = false)
 |-- LimitPersLoan: double (nullable = false)
 |-- LimitCashOut: double (nullable = false)
 |-- CLOSING_BALANCE: double (nullable = false)
 |-- INSTALLMENTSOFFERS: long (nullable = true)
 |-- Emprestimo_Pessoal: long (nullable = true)
 |-- Pagamento_de_Contas: long (nullable = true)
 |-- Parcelamento_Fatura_Automatico: long (nullable = true)
 |-- Parcelamento_Fatura_Facil: long (nullable = true)
 |-- Saque_Parcelado: long (nullable = true)
 |-- Saque_a_Vista: long (nullable = true)
 |-- Overlimit: long (nullable = true)
 |-- Overlimit_Auth: integer (nullable = true)
 |-- NULO: double (nullable = false)
 |-- 0742: double (nullable = false)

In [31]:
dfProdIncl.createOrReplaceTempView("tProdIncl")

In [32]:
dfClusIncl=spark.sql("""SELECT prediction, 
    SUM(CASE WHEN Emprestimo_Pessoal>0 THEN 1 ELSE 0 END)/SUM(1) Emprestimo_Pessoal,
    SUM(CASE WHEN Pagamento_de_Contas>0 THEN 1 ELSE 0 END)/SUM(1) Pagamento_de_Contas, 
    SUM(CASE WHEN Parcelamento_Fatura_Automatico>0 THEN 1 ELSE 0 END)/SUM(1) Parcelamento_Fatura_Automatico, 
    SUM(CASE WHEN Parcelamento_Fatura_Facil>0 THEN 1 ELSE 0 END)/SUM(1) Parcelamento_Fatura_Facil, 
    SUM(CASE WHEN Saque_Parcelado>0 THEN 1 ELSE 0 END)/SUM(1) Saque_Parcelado, 
    SUM(CASE WHEN Saque_a_Vista>0 THEN 1 ELSE 0 END)/SUM(1) Saque_a_Vista,
    SUM(CASE WHEN Overlimit>0 THEN 1 ELSE 0 END)/SUM(1) Overlimit
    from tProdIncl group by prediction order by prediction""")

In [33]:
dfClusIncl.toPandas()

,prediction,Emprestimo_Pessoal,Pagamento_de_Contas,Parcelamento_Fatura_Automatico,Parcelamento_Fatura_Facil,Saque_Parcelado,Saque_a_Vista,Overlimit
0,0,0.015441,0.065705,0.162023,0.233424,0.055689,0.064009,0.350507
1,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
2,2,0.007315,0.044045,0.114402,0.140637,0.017655,0.021666,0.205590
3,3,0.008821,0.048675,0.105959,0.113079,0.020441,0.028907,0.220278
4,4,0.000000,0.000000,0.500000,0.500000,0.000000,0.000000,0.500000
5,5,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
6,6,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.571429
7,7,0.010302,0.050693,0.084192,0.101312,0.019047,0.029200,0.172312
8,8,0.000000,0.090909,0.181818,0.090909,0.090909,0.272727,0.636364
9,9,0.012665,0.047357,0.066630,0.080947,0.023678,0.036344,0.185573


In [34]:
dfTrainRes.select("ACCOUNT","prediction").groupBy("prediction").count().orderBy("prediction").toPandas()

,prediction,count
0,0,301803
1,1,1
2,2,179498
3,3,28228
4,4,2
5,5,1
6,6,7
7,7,13493
8,8,11
9,9,1816


In [35]:
dfImportanceCluster=spark.sql("""
    SELECT prediction, 
    Emprestimo_Pessoal/AVG_Emprestimo_Pessoal AS Emprestimo_Pessoal, 
    Pagamento_de_Contas/AVG_Pagamento_de_Contas AS Pagamento_de_Contas,
    Parcelamento_Fatura_Automatico/AVG_Parcelamento_Fatura_Automatico AS Parcelamento_Fatura_Automatico,
    Parcelamento_Fatura_Facil/AVG_Parcelamento_Fatura_Facil AS Parcelamento_Fatura_Facil,
    Saque_Parcelado/AVG_Saque_Parcelado  AS Saque_Parcelado,
    Saque_a_Vista/AVG_Saque_a_Vista  AS Saque_a_Vista,
    Overlimit/AVG_Overlimit  AS Overlimit
    FROM
    (SELECT prediction, 
    SUM(CASE WHEN Emprestimo_Pessoal>0 THEN 1 ELSE 0 END)/SUM(1) Emprestimo_Pessoal,
    SUM(CASE WHEN Pagamento_de_Contas>0 THEN 1 ELSE 0 END)/SUM(1) Pagamento_de_Contas, 
    SUM(CASE WHEN Parcelamento_Fatura_Automatico>0 THEN 1 ELSE 0 END)/SUM(1) Parcelamento_Fatura_Automatico, 
    SUM(CASE WHEN Parcelamento_Fatura_Facil>0 THEN 1 ELSE 0 END)/SUM(1) Parcelamento_Fatura_Facil, 
    SUM(CASE WHEN Saque_Parcelado>0 THEN 1 ELSE 0 END)/SUM(1) Saque_Parcelado, 
    SUM(CASE WHEN Saque_a_Vista>0 THEN 1 ELSE 0 END)/SUM(1) Saque_a_Vista,
    SUM(CASE WHEN Overlimit>0 THEN 1 ELSE 0 END)/SUM(1) Overlimit
    from tProdIncl group by prediction) CLUSTER CROSS JOIN
    (SELECT  
    SUM(CASE WHEN Emprestimo_Pessoal>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Emprestimo_Pessoal,
    SUM(CASE WHEN Pagamento_de_Contas>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Pagamento_de_Contas, 
    SUM(CASE WHEN Parcelamento_Fatura_Automatico>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Parcelamento_Fatura_Automatico, 
    SUM(CASE WHEN Parcelamento_Fatura_Facil>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Parcelamento_Fatura_Facil, 
    SUM(CASE WHEN Saque_Parcelado>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Saque_Parcelado, 
    SUM(CASE WHEN Saque_a_Vista>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Saque_a_Vista,
    SUM(CASE WHEN Overlimit>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Overlimit
    from tProdIncl)
    WHERE 
    order by prediction""")

In [36]:
dfImportanceCluster.columns[1:]

['Emprestimo_Pessoal',
 'Pagamento_de_Contas',
 'Parcelamento_Fatura_Automatico',
 'Parcelamento_Fatura_Facil',
 'Saque_Parcelado',
 'Saque_a_Vista',
 'Overlimit']

In [37]:
cond = "psf.when" + ".when".join(["(psf.col('" + c + "') == psf.col('max_value'), psf.lit('" + c + "'))" for c in dfImportanceCluster.columns[1:]])
import pyspark.sql.functions as psf
dfImportanceCluster.withColumn("max_value", psf.greatest(*dfImportanceCluster.columns[1:]))\
    .withColumn("MAX", eval(cond))\
    .toPandas()

,prediction,Emprestimo_Pessoal,Pagamento_de_Contas,Parcelamento_Fatura_Automatico,Parcelamento_Fatura_Facil,Saque_Parcelado,Saque_a_Vista,Overlimit,max_value,MAX
0,0,2.462199,2.188608,1.552004,1.555541,3.029857,2.881040,1.823335,3.029857,Saque_Parcelado
1,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.201997,5.201997,Overlimit
2,2,1.166449,1.467122,1.095851,0.937205,0.960546,0.975190,1.069479,1.467122,Pagamento_de_Contas
3,3,1.406630,1.621345,1.014969,0.753562,1.112119,1.301130,1.145884,1.621345,Pagamento_de_Contas
4,4,0.000000,0.000000,4.789459,3.332010,0.000000,0.000000,2.600999,4.789459,Parcelamento_Fatura_Automatico
5,5,0.000000,0.000000,0.000000,0.000000,0.000000,45.010180,0.000000,45.010180,Saque_a_Vista
6,6,0.000000,0.000000,1.368417,0.000000,0.000000,0.000000,2.972570,2.972570,Overlimit
7,7,1.642733,1.688560,0.806466,0.675144,1.036287,1.314312,0.896364,1.688560,Pagamento_de_Contas
8,8,0.000000,3.028142,1.741621,0.605820,4.946098,12.275504,3.310362,12.275504,Saque_a_Vista
9,9,2.019634,1.577435,0.638243,0.539433,1.288273,1.635833,0.965349,2.019634,Emprestimo_Pessoal


In [38]:
#dfTrainRes.write.parquet("/analytics/bc1_clustering/tables/dfTrainRes")

In [39]:
#dfTrainRes=spark.read.parquet("/analytics/bc1_clustering/tables/dfTrainRes")
#dfTrainRes.persist()

#### Test

In [40]:
dfTestBufferFeatureTmp=sparkFeaturePipelineModel.transform(dfTestBuffer).persist()

In [41]:
dfTestBufferFeature=transformer.transform(dfTestBufferFeatureTmp)

In [42]:
dfTestRes=model.transform(dfTestBufferFeature).select(col("ACCOUNTPK").alias("ACCOUNT"),"prediction")

In [43]:
dfTestProdIncl=dfTestRes.join(dfTsys,dfTestRes.ACCOUNT==dfTsys.ACCOUNTPK,'left_outer').fillna(0)

In [44]:
dfTestProdIncl.printSchema()

root
 |-- ACCOUNT: long (nullable = true)
 |-- prediction: integer (nullable = true)
 |-- ACCOUNTPK: long (nullable = true)
 |-- Emprestimo_Pessoal: long (nullable = true)
 |-- Pagamento_de_Contas: long (nullable = true)
 |-- Parcelamento_Fatura_Automatico: long (nullable = true)
 |-- Parcelamento_Fatura_Facil: long (nullable = true)
 |-- Saque_Parcelado: long (nullable = true)
 |-- Saque_a_Vista: long (nullable = true)
 |-- Overlimit: long (nullable = true)



In [45]:
dfTestProdIncl.createOrReplaceTempView("tTestProdIncl")

In [46]:
dfTestClusIncl=spark.sql("""SELECT prediction, 
    SUM(CASE WHEN Emprestimo_Pessoal>0 THEN 1 ELSE 0 END)/SUM(1) Emprestimo_Pessoal,
    SUM(CASE WHEN Pagamento_de_Contas>0 THEN 1 ELSE 0 END)/SUM(1) Pagamento_de_Contas, 
    SUM(CASE WHEN Parcelamento_Fatura_Automatico>0 THEN 1 ELSE 0 END)/SUM(1) Parcelamento_Fatura_Automatico, 
    SUM(CASE WHEN Parcelamento_Fatura_Facil>0 THEN 1 ELSE 0 END)/SUM(1) Parcelamento_Fatura_Facil, 
    SUM(CASE WHEN Saque_Parcelado>0 THEN 1 ELSE 0 END)/SUM(1) Saque_Parcelado, 
    SUM(CASE WHEN Saque_a_Vista>0 THEN 1 ELSE 0 END)/SUM(1) Saque_a_Vista,
    SUM(CASE WHEN Overlimit>0 THEN 1 ELSE 0 END)/SUM(1) Overlimit
    from tTestProdIncl group by prediction order by prediction""")

In [47]:
dfTestClusIncl.toPandas()

,prediction,Emprestimo_Pessoal,Pagamento_de_Contas,Parcelamento_Fatura_Automatico,Parcelamento_Fatura_Facil,Saque_Parcelado,Saque_a_Vista,Overlimit
0,0,0.015582,0.065599,0.163139,0.233526,0.056234,0.064880,0.346775
1,2,0.007347,0.044320,0.113107,0.139863,0.017524,0.021300,0.207119
2,3,0.009147,0.049308,0.107062,0.112844,0.021447,0.028386,0.222323
3,4,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.666667
4,6,0.000000,0.000000,0.000000,0.400000,0.000000,0.000000,0.400000
5,7,0.009010,0.052024,0.087118,0.101213,0.019037,0.028991,0.176560
6,8,0.000000,0.000000,0.400000,0.600000,0.200000,0.000000,0.000000
7,9,0.011809,0.052067,0.075148,0.082126,0.015566,0.033280,0.165862
8,11,0.000000,0.092308,0.169231,0.192308,0.061538,0.053846,0.530769
9,12,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [48]:
dfTestRes.select("ACCOUNT","prediction").groupBy("prediction").count().orderBy("prediction").toPandas()

,prediction,count
0,0,307413
1,2,180891
2,3,28535
3,4,3
4,6,10
5,7,13763
6,8,5
7,9,1863
8,11,130
9,12,1


#### Total

In [49]:
dfTotal=dfTrainRes.union(dfTestRes)

In [50]:
dfTotalProdIncl=dfTotal.join(dfTsys,dfTrainRes.ACCOUNT==dfTsys.ACCOUNTPK,'left_outer').fillna(0)

In [51]:
dfTotalProdIncl.printSchema()

root
 |-- ACCOUNT: long (nullable = true)
 |-- prediction: integer (nullable = true)
 |-- ACCOUNTPK: long (nullable = true)
 |-- Emprestimo_Pessoal: long (nullable = true)
 |-- Pagamento_de_Contas: long (nullable = true)
 |-- Parcelamento_Fatura_Automatico: long (nullable = true)
 |-- Parcelamento_Fatura_Facil: long (nullable = true)
 |-- Saque_Parcelado: long (nullable = true)
 |-- Saque_a_Vista: long (nullable = true)
 |-- Overlimit: long (nullable = true)



In [52]:
dfTotalProdIncl.createOrReplaceTempView("tTotalProdIncl")

In [53]:
dfTotalClusIncl=spark.sql("""SELECT prediction, 
    SUM(CASE WHEN Emprestimo_Pessoal>0 THEN 1 ELSE 0 END)/SUM(1) Emprestimo_Pessoal,
    SUM(CASE WHEN Pagamento_de_Contas>0 THEN 1 ELSE 0 END)/SUM(1) Pagamento_de_Contas, 
    SUM(CASE WHEN Parcelamento_Fatura_Automatico>0 THEN 1 ELSE 0 END)/SUM(1) Parcelamento_Fatura_Automatico, 
    SUM(CASE WHEN Parcelamento_Fatura_Facil>0 THEN 1 ELSE 0 END)/SUM(1) Parcelamento_Fatura_Facil, 
    SUM(CASE WHEN Saque_Parcelado>0 THEN 1 ELSE 0 END)/SUM(1) Saque_Parcelado, 
    SUM(CASE WHEN Saque_a_Vista>0 THEN 1 ELSE 0 END)/SUM(1) Saque_a_Vista,
    SUM(CASE WHEN Overlimit>0 THEN 1 ELSE 0 END)/SUM(1) Overlimit
    from tTotalProdIncl group by prediction order by prediction""")

In [54]:
dfTotalClusIncl.toPandas()

,prediction,Emprestimo_Pessoal,Pagamento_de_Contas,Parcelamento_Fatura_Automatico,Parcelamento_Fatura_Facil,Saque_Parcelado,Saque_a_Vista,Overlimit
0,0,0.015512,0.065652,0.162586,0.233475,0.055964,0.064448,0.348623
1,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
2,2,0.007331,0.044183,0.113752,0.140248,0.017589,0.021482,0.206358
3,3,0.008985,0.048993,0.106513,0.112961,0.020947,0.028645,0.221306
4,4,0.000000,0.200000,0.200000,0.200000,0.000000,0.000000,0.600000
5,5,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
6,6,0.000000,0.000000,0.058824,0.235294,0.000000,0.000000,0.470588
7,7,0.009649,0.051365,0.085669,0.101262,0.019042,0.029095,0.174457
8,8,0.000000,0.062500,0.250000,0.250000,0.125000,0.187500,0.437500
9,9,0.012232,0.049742,0.070943,0.081544,0.019571,0.034792,0.175591


In [55]:
dfTotalClusIncl.createOrReplaceTempView("tTotalClusIncl")

In [56]:
dfOrden=spark.sql("""SELECT prediction,Emprestimo_Pessoal, 
    row_number() over (order by Emprestimo_Pessoal desc) as orden
    from  tTotalClusIncl order by Emprestimo_Pessoal desc""")

In [57]:
dfOrden.toPandas()

,prediction,Emprestimo_Pessoal,orden
0,16,0.083333,1
1,14,0.019447,2
2,0,0.015512,3
3,11,0.015038,4
4,9,0.012232,5
5,15,0.010216,6
6,7,0.009649,7
7,3,0.008985,8
8,19,0.008910,9
9,18,0.008432,10


In [58]:
dfTotal.select("ACCOUNT","prediction").groupBy("prediction").count().orderBy("prediction") \
    .withColumn('Percentage', col('count')/dfTotal.count()).toPandas()

,prediction,count,Percentage
0,0,609216,1.526176e-01
1,1,1,2.505148e-07
2,2,360389,9.028278e-02
3,3,56763,1.421997e-02
4,4,5,1.252574e-06
5,5,1,2.505148e-07
6,6,17,4.258752e-06
7,7,27256,6.828032e-03
8,8,16,4.008237e-06
9,9,3679,9.216440e-04


In [59]:
dfTotalImportanceCluster=spark.sql("""
    SELECT prediction, 
    Emprestimo_Pessoal/AVG_Emprestimo_Pessoal AS Emprestimo_Pessoal, 
    Pagamento_de_Contas/AVG_Pagamento_de_Contas AS Pagamento_de_Contas,
    Parcelamento_Fatura_Automatico/AVG_Parcelamento_Fatura_Automatico AS Parcelamento_Fatura_Automatico,
    Parcelamento_Fatura_Facil/AVG_Parcelamento_Fatura_Facil AS Parcelamento_Fatura_Facil,
    Saque_Parcelado/AVG_Saque_Parcelado  AS Saque_Parcelado,
    Saque_a_Vista/AVG_Saque_a_Vista  AS Saque_a_Vista,
    Overlimit/AVG_Overlimit  AS Overlimit
    FROM
    (SELECT prediction, 
    SUM(CASE WHEN Emprestimo_Pessoal>0 THEN 1 ELSE 0 END)/SUM(1) Emprestimo_Pessoal,
    SUM(CASE WHEN Pagamento_de_Contas>0 THEN 1 ELSE 0 END)/SUM(1) Pagamento_de_Contas, 
    SUM(CASE WHEN Parcelamento_Fatura_Automatico>0 THEN 1 ELSE 0 END)/SUM(1) Parcelamento_Fatura_Automatico, 
    SUM(CASE WHEN Parcelamento_Fatura_Facil>0 THEN 1 ELSE 0 END)/SUM(1) Parcelamento_Fatura_Facil, 
    SUM(CASE WHEN Saque_Parcelado>0 THEN 1 ELSE 0 END)/SUM(1) Saque_Parcelado, 
    SUM(CASE WHEN Saque_a_Vista>0 THEN 1 ELSE 0 END)/SUM(1) Saque_a_Vista,
    SUM(CASE WHEN Overlimit>0 THEN 1 ELSE 0 END)/SUM(1) Overlimit
    from tTotalProdIncl group by prediction) CLUSTER CROSS JOIN
    (SELECT  
    SUM(CASE WHEN Emprestimo_Pessoal>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Emprestimo_Pessoal,
    SUM(CASE WHEN Pagamento_de_Contas>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Pagamento_de_Contas, 
    SUM(CASE WHEN Parcelamento_Fatura_Automatico>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Parcelamento_Fatura_Automatico, 
    SUM(CASE WHEN Parcelamento_Fatura_Facil>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Parcelamento_Fatura_Facil, 
    SUM(CASE WHEN Saque_Parcelado>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Saque_Parcelado, 
    SUM(CASE WHEN Saque_a_Vista>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Saque_a_Vista,
    SUM(CASE WHEN Overlimit>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Overlimit
    from tTotalProdIncl)
    WHERE 
    order by prediction""")

In [60]:
dfOverallPop=spark.sql("""SELECT  
    SUM(CASE WHEN Emprestimo_Pessoal>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Emprestimo_Pessoal,
    SUM(CASE WHEN Pagamento_de_Contas>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Pagamento_de_Contas, 
    SUM(CASE WHEN Parcelamento_Fatura_Automatico>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Parcelamento_Fatura_Automatico, 
    SUM(CASE WHEN Parcelamento_Fatura_Facil>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Parcelamento_Fatura_Facil, 
    SUM(CASE WHEN Saque_Parcelado>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Saque_Parcelado, 
    SUM(CASE WHEN Saque_a_Vista>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Saque_a_Vista,
    SUM(CASE WHEN Overlimit>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Overlimit
    from tTotalProdIncl
    """)

In [61]:
dfOverallPop.toPandas()

,AVG_Emprestimo_Pessoal,AVG_Pagamento_de_Contas,AVG_Parcelamento_Fatura_Automatico,AVG_Parcelamento_Fatura_Facil,AVG_Saque_Parcelado,AVG_Saque_a_Vista,AVG_Overlimit
0,0.006323,0.030089,0.104702,0.150508,0.018516,0.022339,0.191878


In [62]:
dfTotalImportanceCluster.columns[1:]

['Emprestimo_Pessoal',
 'Pagamento_de_Contas',
 'Parcelamento_Fatura_Automatico',
 'Parcelamento_Fatura_Facil',
 'Saque_Parcelado',
 'Saque_a_Vista',
 'Overlimit']

In [63]:
cond = "psf.when" + ".when".join(["(psf.col('" + c + "') == psf.col('max_value'), psf.lit('" + c + "'))" for c in dfTotalImportanceCluster.columns[1:]])
import pyspark.sql.functions as psf
dfTotalImportanceCluster.withColumn("max_value", psf.greatest(*dfTotalImportanceCluster.columns[1:]))\
    .withColumn("MAX", eval(cond))\
    .toPandas()

,prediction,Emprestimo_Pessoal,Pagamento_de_Contas,Parcelamento_Fatura_Automatico,Parcelamento_Fatura_Facil,Saque_Parcelado,Saque_a_Vista,Overlimit,max_value,MAX
0,0,2.453033,2.181889,1.552843,1.551252,3.022444,2.884997,1.816906,3.022444,Saque_Parcelado
1,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.211657,5.211657,Overlimit
2,2,1.159322,1.468388,1.086435,0.931835,0.949950,0.961645,1.075465,1.468388,Pagamento_de_Contas
3,3,1.420848,1.628257,1.017296,0.750532,1.131275,1.282296,1.153372,1.628257,Pagamento_de_Contas
4,4,0.000000,6.646874,1.910180,1.328835,0.000000,0.000000,3.126994,6.646874,Pagamento_de_Contas
5,5,0.000000,0.000000,0.000000,0.000000,0.000000,44.764447,0.000000,44.764447,Saque_a_Vista
6,6,0.000000,0.000000,0.561818,1.563335,0.000000,0.000000,2.452544,2.452544,Overlimit
7,7,1.525937,1.707078,0.818218,0.672803,1.028388,1.302400,0.909210,1.707078,Pagamento_de_Contas
8,8,0.000000,2.077148,2.387725,1.661044,6.750900,8.393334,2.280100,8.393334,Saque_a_Vista
9,9,1.934308,1.653137,0.677571,0.541792,1.056950,1.557447,0.915121,1.934308,Emprestimo_Pessoal


### Cluster Analysis

#### KS2 Evaluation

In [64]:
dfOrden.persist().toPandas()

,prediction,Emprestimo_Pessoal,orden
0,16,0.083333,1
1,14,0.019447,2
2,0,0.015512,3
3,11,0.015038,4
4,9,0.012232,5
5,15,0.010216,6
6,7,0.009649,7
7,3,0.008985,8
8,19,0.008910,9
9,18,0.008432,10


In [65]:
dfValidacionNov=spark.read.parquet("/analytics/bc1_clustering/tables/dfValidacionNov")

In [66]:
dfValidacionNov.printSchema()

root
 |-- TRANSACTIONSERNO: long (nullable = true)
 |-- ACCOUNTSERNOFK: long (nullable = true)
 |-- TYPESERNO_GLEDGER: long (nullable = true)
 |-- VALUEDATE: string (nullable = true)
 |-- I013_TRXN_DATE: string (nullable = true)
 |-- POSTDATE: string (nullable = true)
 |-- STGENERAL: string (nullable = true)
 |-- REFDATE: timestamp (nullable = true)
 |-- FG_CURRENT: integer (nullable = true)
 |-- TM_INSERT: timestamp (nullable = true)
 |-- PRODUTO: string (nullable = true)



In [67]:
dfValidacionNov.where(col('PRODUTO').like('Empr%')).select('ACCOUNTSERNOFK')

DataFrame[ACCOUNTSERNOFK: bigint]

In [68]:
dfEvalTmp1=dfTotalProdIncl.select('ACCOUNT','prediction','Emprestimo_Pessoal') \
    .join(dfValidacionNov.where(col('PRODUTO').like('Empr%')).select('ACCOUNTSERNOFK'),dfValidacionNov.ACCOUNTSERNOFK==dfTotalProdIncl.ACCOUNT,'left_outer') \
    .withColumn('EP',when(col('ACCOUNTSERNOFK').isNull(),0).otherwise(1)) 

In [69]:
dfEvalTmp1.persist()

DataFrame[ACCOUNT: bigint, prediction: int, Emprestimo_Pessoal: bigint, ACCOUNTSERNOFK: bigint, EP: int]

In [70]:
dfEvalTmp1.groupBy('EP','Emprestimo_Pessoal').count().show()

+---+------------------+-------+
| EP|Emprestimo_Pessoal|  count|
+---+------------------+-------+
|  1|                 1|     23|
|  0|                 0|3965535|
|  1|                 0|   1003|
|  0|                 1|  25219|
+---+------------------+-------+



In [71]:
dfEvalTmp1.printSchema()

root
 |-- ACCOUNT: long (nullable = true)
 |-- prediction: integer (nullable = true)
 |-- Emprestimo_Pessoal: long (nullable = true)
 |-- ACCOUNTSERNOFK: long (nullable = true)
 |-- EP: integer (nullable = false)



In [72]:
dfEval=dfEvalTmp1.join(dfOrden,dfEvalTmp1.prediction==dfOrden.prediction).drop(dfOrden.prediction).drop(dfOrden.Emprestimo_Pessoal) \
    .drop(dfEvalTmp1.ACCOUNTSERNOFK)

In [73]:
dfEval=dfEvalTmp1.join(dfOrden,dfEvalTmp1.prediction==dfOrden.prediction) \
    .select('ACCOUNT',dfEvalTmp1.prediction, 'EP','orden')

In [74]:
dfEval.printSchema()

root
 |-- ACCOUNT: long (nullable = true)
 |-- prediction: integer (nullable = true)
 |-- EP: integer (nullable = false)
 |-- orden: integer (nullable = true)



In [75]:
Positives=[i.orden for i in dfEval.where(col('EP')==1).select('orden').collect()]

In [76]:
Negatives=[i.orden for i in dfEval.where(col('EP')==0).select('orden').collect()]

In [77]:
stats.ks_2samp(Positives,Negatives)

Ks_2sampResult(statistic=0.40913170159294948, pvalue=1.0360965910822277e-150)

In [78]:
dfEval.groupBy('prediction').agg((sum('EP')/count('*')).alias('porcentaje')).orderBy('porcentaje',ascending=False).toPandas()

,prediction,porcentaje
0,15,0.001513
1,14,0.001249
2,0,0.000671
3,7,0.000660
4,9,0.000544
5,18,0.000493
6,2,0.000333
7,3,0.000299
8,13,0.000104
9,8,0.000000


#### Variable analysis

In [ ]:
dfAnVar=dfBufferOrig.join(dfTotal,dfTotal.ACCOUNT==dfBufferOrig.ACCOUNTPK).drop(dfTotal.ACCOUNT).persist()

In [ ]:
dfAnVar.printSchema()

In [ ]:
dfAnVar.select('BALANCE').collect()

In [ ]:
total=dfAnVar.select('BALANCE').collect()

In [ ]:
cluster=dfAnVar.where(col('prediction')==4).select('BALANCE').collect()

In [ ]:
stats.f_oneway(total,cluster)

In [ ]:
dfTrainRes.printSchema()

#### Products

In [ ]:
#dfAnalysis=spark.read.parquet("/analytics/bc1_clustering/tables/dfAnalysisBin")
dfAnalysis=spark.read.parquet("/analytics/bc1_clustering/tables/dfAnalysis11Dic")

In [ ]:
dfAnalysis.printSchema()

In [ ]:
dfAnTmp1=dfAnalysis.join(dfTrainRes,dfTrainRes.ACCOUNT==dfAnalysis.ACCOUNTPK)

In [ ]:
dfAnTmp1.printSchema()

In [ ]:
dfAnTmp1.createOrReplaceTempView('tdfAnTmp1')

In [ ]:
dfSectorImportance=spark.sql("""SELECT * FROM (
    SELECT prediction, SECTOR, IMPORTANCE, 
    row_number() over (partition by prediction order by IMPORTANCE desc) as orden FROM 
    (
    SELECT prediction, SECTOR, AVGPRED/AVGTOTAL AS IMPORTANCE FROM 
    (SELECT PRF_DS_SETOR as SECTOR, SUM(PURCHASES)/COUNT(DISTINCT ACCOUNT) AS AVGTOTAL
    FROM tdfAnTmp1 GROUP BY PRF_DS_SETOR ORDER BY SECTOR) A,
    (SELECT prediction, PRF_DS_SETOR as SECTORPRED, SUM(PURCHASES)/COUNT(DISTINCT ACCOUNT) AS AVGPRED
    FROM tdfAnTmp1 GROUP BY prediction, PRF_DS_SETOR ORDER BY SECTORPRED) B
    WHERE SECTOR=SECTORPRED ORDER BY prediction, SECTOR) MAIN
    ORDER BY prediction, orden) FINAL
    WHERE orden=1""")
dfSectorImportance.printSchema()

In [ ]:
dfSectorImportance.toPandas()

#### Demographic

In [ ]:
#LECTURA DE TABLA PARQUET
dfPeople=spark.read.parquet("/analytics/bc1_clustering/tables/dfPeople")
dfPeople=dfPeople.withColumnRenamed("SERNO","PEOPLESERNO").withColumn('YOB',(dfPeople["DOB"]/10000).cast('integer'))
dfAccounts=spark.read.parquet("/analytics/bc1_clustering/tables/dfAccountsRefdate")
dfAccounts=dfAccounts.withColumnRenamed("SERNO","ACCOUNTSERNO").withColumnRenamed("PEOPLESERNO","PEOPLESERNOFK")

In [ ]:
dfDemographic=dfTrainRes.join(dfAccounts,dfAccounts.ACCOUNTSERNO==dfTrainRes.ACCOUNT) \
    .join(dfPeople,dfPeople.PEOPLESERNO==dfAccounts.PEOPLESERNOFK) \
    .select('ACCOUNTSERNO','prediction','PEOPLESERNO','SEX','MARITAL_STATUS','YOB').persist()

In [ ]:
dfDemographic.createOrReplaceTempView("tDemographic")

In [ ]:
dfAge=spark.sql("""SELECT prediction, round(AVG(YOB),0) as cuenta from tDemographic group by prediction order by prediction""")
dfAge.toPandas()

In [ ]:
dfSex=spark.sql("""SELECT A.*, B.total, round(cuenta/total,2) as porcentaje FROM 
    (SELECT prediction, SEX, COUNT(*) as cuenta from tDemographic group by prediction, SEX) A,
    (SELECT prediction, COUNT(*) as total from tDemographic group by prediction) B
    WHERE A.prediction=B.prediction order by prediction, SEX""")
dfSex.toPandas()

In [ ]:
dfMarital=spark.sql("""SELECT A.*, B.total, round(cuenta/total,2) as porcentaje FROM 
    (SELECT prediction, MARITAL_STATUS, COUNT(*) as cuenta from tDemographic group by prediction, MARITAL_STATUS) A,
    (SELECT prediction, COUNT(*) as total from tDemographic group by prediction) B
    WHERE A.prediction=B.prediction order by prediction, MARITAL_STATUS""")
dfMarital.toPandas()

### Model saving

In [ ]:
#Iter2 -> grupo, part, dinero, 6*IQR

In [ ]:
model.save("/analytics/bc1_clustering/models/model02EneGrpNWVal")

In [ ]:
sparkFeaturePipelineModel.save("/analytics/bc1_clustering/models/pipeline02EneGrpNWVal")

In [ ]:
transformer.save("/analytics/bc1_clustering/models/scaleFinancial02EneGrpNWVal")

In [ ]:
dfOrden.write.parquet("/analytics/bc1_clustering/tables/dfOrden02EneGrpNWVal")

In [ ]:
dfOrden=spark.read.parquet("/analytics/bc1_clustering/tables/dfOrden02EneGrpNWVal")

In [ ]:
transformer=ElementwiseProduct.load("/analytics/bc1_clustering/models/scaleFinancial02EneGrpNWVal")

In [ ]:
sparkFeaturePipelineModel=PipelineModel.load("/analytics/bc1_clustering/models/pipeline02EneGrpNWVal")

In [ ]:
model=BisectingKMeansModel.load("/analytics/bc1_clustering/models/model02EneGrpNWVal")